In [1]:
# pip install opencv-python
# pip install face_recognition
# pip install dlib
# pip install pandas
# pip install numpy

In [2]:
import os
import face_recognition
import pickle
import cv2
import face_recognition
import pickle
import datetime
import sqlite3
import pandas as pd
import numpy as np

In [3]:
# مسار قاعدة البيانات
dataset_dir = './content'
encodings_file = 'encodings.pickle'

# قوائم لحفظ البيانات
known_encodings = []
known_names = []

# قراءة الصور وتشفيرها
for person_name in os.listdir(dataset_dir):
    person_dir = os.path.join(dataset_dir, person_name)
    if not os.path.isdir(person_dir):
        continue
    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)
        image = face_recognition.load_image_file(image_path)
        try:
            encodings = face_recognition.face_encodings(image)[0]
        except IndexError:
            print(f"لم يتم العثور على وجه في الصورة {image_path}")
            continue
        known_encodings.append(encodings)
        known_names.append(person_name)

In [ ]:
# حفظ التشفيرات
data = {"encodings": known_encodings, "names": known_names}
with open(encodings_file, "wb") as f:
    f.write(pickle.dumps(data))

print("تم إنشاء قاعدة البيانات بنجاح!")

In [5]:
# تحميل التشفيرات المعروفة
with open('encodings.pickle', 'rb') as f:
    data = pickle.load(f)

In [6]:
# الوصول إلى الكاميرا
video_capture = cv2.VideoCapture(2)

In [7]:
# قائمة لتخزين الأسماء المكتشفة
recognized_names = []

In [ ]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # تحويل الصورة من BGR إلى RGB
    rgb_frame = frame[:, :, ::-1]

    # اكتشاف مواقع الوجوه
    face_locations = face_recognition.face_locations(rgb_frame)
    # تشفير الوجوه
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # مقارنة الوجوه
        matches = face_recognition.compare_faces(data["encodings"], face_encoding)
        name = "Unknown"

        # العثور على أقرب تطابق
        face_distances = face_recognition.face_distance(data["encodings"], face_encoding)
        best_match_index = face_distances.argmin()

        if matches[best_match_index]:
            name = data["names"][best_match_index]

        # رسم مربع حول الوجه
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        # كتابة الاسم
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # تسجيل الوقت والاسم إذا لم يتم تسجيله من قبل
        if name != "Unknown" and name not in recognized_names:
            recognized_names.append(name)
            time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{name} وصل في {time_stamp}")
            # يمكنك هنا حفظ البيانات في قاعدة بيانات أو ملف

    # عرض الفيديو
    cv2.imshow('Video', frame)

    # الخروج عند الضغط على 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

# تحرير الموارد
video_capture.release()
cv2.destroyAllWindows()


In [ ]:
# تحميل التشفيرات المعروفة
with open('encodings.pickle', 'rb') as f:
    data = pickle.load(f)

# الوصول إلى الكاميرا
video_capture = cv2.VideoCapture(2)

# إنشاء القاموس للتتبع
trackers = {}
tracker_id = 0

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    rgb_frame = frame[:, :, ::-1]

    # اكتشاف مواقع الوجوه
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(data["encodings"], face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(data["encodings"], face_encoding)
        best_match_index = face_distances.argmin()

        if matches[best_match_index]:
            name = data["names"][best_match_index]

        # إنشاء متتبع جديد
        tracker = cv2.TrackerCSRT_create()
        bbox = (left, top, right - left, bottom - top)
        tracker.init(frame, bbox)
        trackers[tracker_id] = {'tracker': tracker, 'name': name}
        tracker_id += 1

    # تحديث وتتبع الجميع
    for tid in list(trackers.keys()):
        success, bbox = trackers[tid]['tracker'].update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, trackers[tid]['name'], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        else:
            # إذا فشل التتبع، نحذفه
            del trackers[tid]

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


import sqlite3

# إنشاء الاتصال بقاعدة البيانات
conn = sqlite3.connect('employee_data.db')
c = conn.cursor()

# إنشاء جدول
c.execute('''CREATE TABLE IF NOT EXISTS movements
             (name TEXT, time TEXT, event TEXT)''')

# عند تسجيل حدث
def log_event(name, event):
    time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO movements VALUES (?, ?, ?)", (name, time_stamp, event))
    conn.commit()

# مثال على الاستخدام
log_event('Ahmed', 'Entered')
